In [3]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.5/210.5 KB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 8.7 MB/s eta 0:00:00


In [17]:
!pip install eli5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 KB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107748 sha256=3f6ce0628754c686fbfce550c59b716cdbda97724eb0ef6b9ae86f10527f2e8f
  Stored in directory: /root/.cache/pip/wheels/85/ac/25/ffcd87ef8f9b1eec324fdf339359be71f22612459d8c75d89c
Successfully built eli5


In [4]:
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.metrics import f1_score

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.feature_selection import SequentialFeatureSelector, RFE, SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator


import optuna

import pickle

**Данный ноутбук содержит следующие части:**

*   label encoding для целевой переменной
*   генерация дополнительных фичей
*   перевод фичи с текстом в кодированную матрицу с помощью tf-df
*   эксперименты с моделью Random Forest:
*      подбор фичей на части датасета
*      подбор оптимальных гиперпараметров на части датасета


In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Загрузим наш  датасет (200 обьектов ) как экперементальный и  для теста ( 50 обьектов). Так же в изначальный датасет были добавлены дополнительные признаки, такие как:
*   Year (год)
*   Month
*   Day
*   Day of week
*   season

Таргет переменную (topic)была закодирована label encording





In [10]:
# df = pd.read_csv('/content/drive/Othercomputers/My Laptop/Project_1_HSE/clear_data.csv')
# df.drop('Unnamed: 0', axis=1, inplace=True)

X_train = pd.read_csv('/content/drive/Othercomputers/My Laptop/Project_1_HSE/данные/train_200k_texts.csv')
##Y_train = pd.read_csv('/content/drive/Othercomputers/My Laptop/Project_1_HSE/данные/ytrain_200k_texts.csv')
X_test = pd.read_csv('/content/drive/Othercomputers/My Laptop/Project_1_HSE/данные/test_50k_texts.csv')
#Y_test = pd.read_csv('/content/drive/Othercomputers/My Laptop/Project_1_HSE/данные/ytest_50k_texts.csv')

In [31]:
X_train.head()

,Unnamed: 0,date,url,topic,tags,text_str,topic_le,title_lemmas,year,month,day,date_enc,seanos,day_of_week,dummy_weekday
0,337202,2010-07-12,https://lenta.ru/news/2010/07/12/explosion/,Россия,Все,запад москва квартира этажный дом происходить ...,20,дом запад москва происходить взрыв газ,2010,7,12,3966,3,0,1
1,460023,2013-03-29,https://lenta.ru/news/2013/03/29/samogon/,Бывший СССР,NaN,украина предлагать вводить уголовный ответстве...,3,украинский самогонщик пригрозить уголовный отв...,2013,3,29,4958,2,4,1
2,340676,2010-08-09,https://lenta.ru/news/2010/08/09/aliev/,Интернет и СМИ,Все,улица москва близкий время появляться социальн...,7,андрей тихонов сергей игнашевич рекламировать ...,2010,8,9,3993,3,0,1
3,88483,2004-02-28,https://lenta.ru/news/2004/02/28/chita/,Россия,Все,ход расследование взрыв читинский кафе онон за...,20,чита задерживать предполагать виновник взрыв кафе,2004,2,28,1642,1,5,0
4,939735,2021-06-18,https://lenta.ru/news/2021/06/18/iron/,Ценности,Явления,эксперт раскрывать альтернативный способ погла...,25,эксперт раскрывать способ погладить одежда утюг,2021,6,18,7957,3,4,1


In [32]:
X_train['topic'].value_counts()

Россия               43390
Мир                  35240
Экономика            21982
Бывший СССР          16878
Спорт                16428
Наука и техника      13610
Культура             12918
Интернет и СМИ       11664
Из жизни              8066
Силовые структуры     7792
Путешествия           4096
Ценности              4018
Дом                   3984
Среда обитания        2809
Бизнес                1337
Моя страна            1103
Нацпроекты             851
69-я параллель         604
Забота о себе          571
Крым                   120
Культпросвет            61
Легпром                 21
Библиотека              11
Оружие                   2
Name: topic, dtype: int64

In [33]:
X_test['topic'].value_counts()

Россия               8678
Мир                  7048
Экономика            4396
Бывший СССР          3376
Спорт                3286
Наука и техника      2722
Культура             2584
Интернет и СМИ       2333
Из жизни             1613
Силовые структуры    1558
Путешествия           819
Ценности              804
Дом                   797
Среда обитания        562
Бизнес                268
Моя страна            221
Нацпроекты            170
69-я параллель        121
Забота о себе         114
Крым                   24
Культпросвет           12
Легпром                 4
Библиотека              2
Name: topic, dtype: int64

In [7]:
X_train.shape

(207556, 15)

In [8]:
X_test.shape

(41512, 15)

два топика ('Оружие', 'Библиотека') редко встречаются и наверняка их модель не предскажет - удалим их.

In [13]:
X_train = X_train[~(X_train['topic'].isin(['Библиотека', 'Оружие']))]
X_train.reset_index(drop=True, inplace=True)
X_test = X_test[~(X_test['topic'].isin(['Библиотека', 'Оружие']))]
X_test.reset_index(drop=True, inplace=True)

In [11]:
X_train.head()

,Unnamed: 0,date,url,topic,tags,text_str,topic_le,title_lemmas,year,month,day,date_enc,seanos,day_of_week,dummy_weekday
0,337202,2010-07-12,https://lenta.ru/news/2010/07/12/explosion/,Россия,Все,запад москва квартира этажный дом происходить ...,20,дом запад москва происходить взрыв газ,2010,7,12,3966,3,0,1
1,460023,2013-03-29,https://lenta.ru/news/2013/03/29/samogon/,Бывший СССР,NaN,украина предлагать вводить уголовный ответстве...,3,украинский самогонщик пригрозить уголовный отв...,2013,3,29,4958,2,4,1
2,340676,2010-08-09,https://lenta.ru/news/2010/08/09/aliev/,Интернет и СМИ,Все,улица москва близкий время появляться социальн...,7,андрей тихонов сергей игнашевич рекламировать ...,2010,8,9,3993,3,0,1
3,88483,2004-02-28,https://lenta.ru/news/2004/02/28/chita/,Россия,Все,ход расследование взрыв читинский кафе онон за...,20,чита задерживать предполагать виновник взрыв кафе,2004,2,28,1642,1,5,0
4,939735,2021-06-18,https://lenta.ru/news/2021/06/18/iron/,Ценности,Явления,эксперт раскрывать альтернативный способ погла...,25,эксперт раскрывать способ погладить одежда утюг,2021,6,18,7957,3,4,1


In [6]:
Y_train = X_train['topic_le']
Y_test = X_test['topic_le']
Y_train.value_counts()

20    43390
13    35240
27    21982
3     16878
23    16428
16    13610
10    12918
7     11664
6      8066
21     7792
19     4096
25     4018
4      3984
24     2809
2      1337
15     1103
17      851
0       604
5       571
8       120
9        61
11       21
Name: topic_le, dtype: int64

**Text data transformation** 

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(ngram_range=(1,1), min_df = 0.0001, max_df = 0.8)
vec.fit(X_train['text_str'])

TfidfVectorizer(max_df=0.8, min_df=0.0001)

In [47]:
def apply_vectorizer(df, vectorizer=vec):
  bow = vectorizer.transform(df['text_str'])

  include_columns = [
    'year', 'day_of_week', 'seanos'
  ]
  return scipy.sparse.hstack([
      bow, 
      scipy.sparse.csr_matrix(df[include_columns].astype('float32').values)
  ]).tocsr()

In [48]:
full_X_train = apply_vectorizer(X_train, vec)

In [49]:
full_X_train

<207543x34201 sparse matrix of type '<class 'numpy.float64'>'
	with 20692525 stored elements in Compressed Sparse Row format>

In [50]:
full_X_test = apply_vectorizer(X_test, vec)

In [51]:
full_X_test

<41510x34201 sparse matrix of type '<class 'numpy.float64'>'
	with 4160010 stored elements in Compressed Sparse Row format>

**Прошлые эксперименты (без optuna)**

Используем матрицу признаков,где для текста ( df_min = 0.001)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Создаём модель леса из 400 деревьев и глубиной 60
model = RandomForestClassifier(
  n_estimators=400,
  max_depth = 60,
  #min_samples_split = 5,
  bootstrap=True,
  max_features='sqrt',
  verbose=2,
  n_jobs=-1
)

# Обучаем на тренировочных данных
model.fit(full_X_train, Y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 400building tree 2 of 400

building tree 3 of 400
building tree 4 of 400
building tree 5 of 400
building tree 6 of 400
building tree 7 of 400
building tree 8 of 400
building tree 9 of 400
building tree 10 of 400
building tree 11 of 400
building tree 12 of 400
building tree 13 of 400
building tree 14 of 400
building tree 15 of 400
building tree 16 of 400
building tree 17 of 400
building tree 18 of 400
building tree 19 of 400
building tree 20 of 400
building tree 21 of 400
building tree 22 of 400
building tree 23 of 400
building tree 24 of 400
building tree 25 of 400
building tree 26 of 400
building tree 27 of 400
building tree 28 of 400
building tree 29 of 400
building tree 30 of 400
building tree 31 of 400
building tree 32 of 400
building tree 33 of 400
building tree 34 of 400
building tree 35 of 400
building tree 36 of 400
building tree 37 of 400
building tree 38 of 400


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  8.3min


building tree 39 of 400
building tree 40 of 400
building tree 41 of 400
building tree 42 of 400
building tree 43 of 400
building tree 44 of 400
building tree 45 of 400
building tree 46 of 400
building tree 47 of 400
building tree 48 of 400
building tree 49 of 400
building tree 50 of 400
building tree 51 of 400
building tree 52 of 400
building tree 53 of 400
building tree 54 of 400
building tree 55 of 400
building tree 56 of 400
building tree 57 of 400
building tree 58 of 400
building tree 59 of 400
building tree 60 of 400
building tree 61 of 400
building tree 62 of 400
building tree 63 of 400
building tree 64 of 400
building tree 65 of 400
building tree 66 of 400
building tree 67 of 400
building tree 68 of 400
building tree 69 of 400
building tree 70 of 400
building tree 71 of 400
building tree 72 of 400
building tree 73 of 400
building tree 74 of 400
building tree 75 of 400
building tree 76 of 400
building tree 77 of 400
building tree 78 of 400
building tree 79 of 400
building tree 80

[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 32.7min


building tree 160 of 400
building tree 161 of 400
building tree 162 of 400
building tree 163 of 400
building tree 164 of 400
building tree 165 of 400
building tree 166 of 400
building tree 167 of 400
building tree 168 of 400
building tree 169 of 400
building tree 170 of 400
building tree 171 of 400
building tree 172 of 400
building tree 173 of 400
building tree 174 of 400
building tree 175 of 400
building tree 176 of 400
building tree 177 of 400
building tree 178 of 400
building tree 179 of 400
building tree 180 of 400
building tree 181 of 400
building tree 182 of 400
building tree 183 of 400
building tree 184 of 400
building tree 185 of 400
building tree 186 of 400
building tree 187 of 400
building tree 188 of 400
building tree 189 of 400
building tree 190 of 400
building tree 191 of 400
building tree 192 of 400
building tree 193 of 400
building tree 194 of 400
building tree 195 of 400
building tree 196 of 400
building tree 197 of 400
building tree 198 of 400
building tree 199 of 400


[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed: 74.1min


building tree 363 of 400
building tree 364 of 400
building tree 365 of 400
building tree 366 of 400
building tree 367 of 400
building tree 368 of 400
building tree 369 of 400
building tree 370 of 400
building tree 371 of 400
building tree 372 of 400
building tree 373 of 400
building tree 374 of 400
building tree 375 of 400
building tree 376 of 400
building tree 377 of 400
building tree 378 of 400
building tree 379 of 400
building tree 380 of 400
building tree 381 of 400
building tree 382 of 400
building tree 383 of 400
building tree 384 of 400
building tree 385 of 400
building tree 386 of 400
building tree 387 of 400
building tree 388 of 400
building tree 389 of 400
building tree 390 of 400
building tree 391 of 400
building tree 392 of 400
building tree 393 of 400
building tree 394 of 400
building tree 395 of 400
building tree 396 of 400
building tree 397 of 400
building tree 398 of 400
building tree 399 of 400
building tree 400 of 400


[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed: 81.6min finished


RandomForestClassifier(max_depth=60, max_features='sqrt', n_estimators=400,
                       n_jobs=-1, verbose=2)

На трейне скор 0.96. Модель скорее всего переобучилась

In [ ]:
from sklearn.metrics import f1_score
pred_train = model.predict(full_X_train)
score_train = f1_score(pred_train, Y_train, sample_weight=None, average="micro")
score_train

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    4.1s
[Parallel(n_jobs=2)]: Done 158 tasks      | elapsed:   17.3s
[Parallel(n_jobs=2)]: Done 361 tasks      | elapsed:   39.1s
[Parallel(n_jobs=2)]: Done 400 out of 400 | elapsed:   43.3s finished


0.9695677522248402

На тесте скор - 0.73 и есть редкие классы которые предсказать не может либо они близки с другими классами по смыслу

In [ ]:

pred = model.predict(full_X_test)
score = f1_score(pred, Y_test, sample_weight=None, average="micro")
score

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    1.3s
[Parallel(n_jobs=2)]: Done 158 tasks      | elapsed:    4.3s
[Parallel(n_jobs=2)]: Done 361 tasks      | elapsed:    9.1s
[Parallel(n_jobs=2)]: Done 400 out of 400 | elapsed:   10.0s finished


0.7323295591423753

In [ ]:
dict_topic = dict(zip(X_train.topic, X_train.topic_le))
keys = list(dict_topic.keys())
keys.sort()
dict_topic_new = {}
for key in keys:
  dict_topic_new[key] = dict_topic[key]
dict_topic_new

{'69-я параллель': 0,
 'Бизнес': 2,
 'Бывший СССР': 3,
 'Дом': 4,
 'Забота о себе': 5,
 'Из жизни': 6,
 'Интернет и СМИ': 7,
 'Крым': 8,
 'Культпросвет ': 9,
 'Культура': 10,
 'Легпром': 11,
 'Мир': 13,
 'Моя страна': 15,
 'Наука и техника': 16,
 'Нацпроекты': 17,
 'Путешествия': 19,
 'Россия': 20,
 'Силовые структуры': 21,
 'Спорт': 23,
 'Среда обитания': 24,
 'Ценности': 25,
 'Экономика': 27}

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn import metrics

print (metrics.classification_report (Y_train, pred_train,target_names = dict_topic))

                   precision    recall  f1-score   support

           Россия       1.00      0.68      0.81       604
      Бывший СССР       1.00      0.96      0.98      1337
   Интернет и СМИ       1.00      0.97      0.98     16878
         Ценности       1.00      0.93      0.96      3984
         Культура       1.00      0.54      0.71       571
  Наука и техника       1.00      0.87      0.93      8066
   Среда обитания       1.00      0.95      0.97     11664
            Спорт       1.00      0.78      0.87       120
         Из жизни       1.00      0.80      0.89        61
              Мир       1.00      0.97      0.99     12918
Силовые структуры       1.00      0.86      0.92        21
        Экономика       0.97      0.98      0.97     35240
       Моя страна       1.00      0.76      0.86      1103
              Дом       1.00      0.97      0.99     13610
           Бизнес       1.00      0.98      0.99       851
       Нацпроекты       1.00      0.94      0.97      4

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn import metrics

print (metrics.classification_report(Y_test, pred,target_names = dict_topic))

                   precision    recall  f1-score   support

           Россия       0.00      0.00      0.00       121
      Бывший СССР       0.00      0.00      0.00       268
   Интернет и СМИ       0.77      0.71      0.74      3376
         Ценности       0.89      0.53      0.66       797
         Культура       1.00      0.01      0.02       114
  Наука и техника       0.83      0.37      0.51      1613
   Среда обитания       0.78      0.40      0.53      2333
            Спорт       0.00      0.00      0.00        24
         Из жизни       0.00      0.00      0.00        12
              Мир       0.81      0.81      0.81      2584
Силовые структуры       0.00      0.00      0.00         4
        Экономика       0.70      0.83      0.76      7048
       Моя страна       1.00      0.00      0.01       221
              Дом       0.81      0.77      0.79      2722
           Бизнес       0.99      0.79      0.88       170
       Нацпроекты       0.89      0.55      0.68       

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Попробуем те же гиперпараметры но уменьшим порог в матрице
 tf-df ( df_min = 0.0001). Те слов станет больше - что возможно поможет предсказать редкие классы.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Создаём модель леса из 400 деревьев
model = RandomForestClassifier(
  n_estimators=400,
  max_depth = 60,
  #min_samples_split = 5,
  bootstrap=True,
  max_features='sqrt',
  random_state=123,
  verbose=2,
  n_jobs=-1
)

# Обучаем на тренировочных данных
model.fit(full_X_train, Y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 400
building tree 2 of 400
building tree 3 of 400
building tree 4 of 400
building tree 5 of 400
building tree 6 of 400
building tree 7 of 400
building tree 8 of 400
building tree 9 of 400
building tree 10 of 400
building tree 11 of 400
building tree 12 of 400
building tree 13 of 400
building tree 14 of 400
building tree 15 of 400
building tree 16 of 400
building tree 17 of 400
building tree 18 of 400
building tree 19 of 400
building tree 20 of 400
building tree 21 of 400
building tree 22 of 400
building tree 23 of 400
building tree 24 of 400
building tree 25 of 400
building tree 26 of 400
building tree 27 of 400
building tree 28 of 400
building tree 29 of 400
building tree 30 of 400
building tree 31 of 400
building tree 32 of 400
building tree 33 of 400
building tree 34 of 400
building tree 35 of 400
building tree 36 of 400


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.4min


building tree 37 of 400
building tree 38 of 400
building tree 39 of 400
building tree 40 of 400
building tree 41 of 400
building tree 42 of 400
building tree 43 of 400
building tree 44 of 400
building tree 45 of 400
building tree 46 of 400
building tree 47 of 400
building tree 48 of 400
building tree 49 of 400
building tree 50 of 400
building tree 51 of 400
building tree 52 of 400
building tree 53 of 400
building tree 54 of 400
building tree 55 of 400
building tree 56 of 400
building tree 57 of 400
building tree 58 of 400
building tree 59 of 400
building tree 60 of 400
building tree 61 of 400
building tree 62 of 400
building tree 63 of 400
building tree 64 of 400
building tree 65 of 400
building tree 66 of 400
building tree 67 of 400
building tree 68 of 400
building tree 69 of 400
building tree 70 of 400
building tree 71 of 400
building tree 72 of 400
building tree 73 of 400
building tree 74 of 400
building tree 75 of 400
building tree 76 of 400
building tree 77 of 400
building tree 78

[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 10.6min


building tree 158 of 400
building tree 159 of 400
building tree 160 of 400
building tree 161 of 400
building tree 162 of 400
building tree 163 of 400
building tree 164 of 400
building tree 165 of 400
building tree 166 of 400
building tree 167 of 400
building tree 168 of 400
building tree 169 of 400
building tree 170 of 400
building tree 171 of 400
building tree 172 of 400
building tree 173 of 400
building tree 174 of 400
building tree 175 of 400
building tree 176 of 400
building tree 177 of 400
building tree 178 of 400
building tree 179 of 400
building tree 180 of 400
building tree 181 of 400
building tree 182 of 400
building tree 183 of 400
building tree 184 of 400
building tree 185 of 400
building tree 186 of 400
building tree 187 of 400
building tree 188 of 400
building tree 189 of 400
building tree 190 of 400
building tree 191 of 400
building tree 192 of 400
building tree 193 of 400
building tree 194 of 400
building tree 195 of 400
building tree 196 of 400
building tree 197 of 400


[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 24.2min


building tree 362 of 400
building tree 363 of 400
building tree 364 of 400
building tree 365 of 400
building tree 366 of 400
building tree 367 of 400
building tree 368 of 400
building tree 369 of 400
building tree 370 of 400
building tree 371 of 400
building tree 372 of 400
building tree 373 of 400
building tree 374 of 400
building tree 375 of 400
building tree 376 of 400
building tree 377 of 400
building tree 378 of 400
building tree 379 of 400
building tree 380 of 400
building tree 381 of 400
building tree 382 of 400
building tree 383 of 400
building tree 384 of 400
building tree 385 of 400
building tree 386 of 400
building tree 387 of 400
building tree 388 of 400
building tree 389 of 400
building tree 390 of 400
building tree 391 of 400
building tree 392 of 400
building tree 393 of 400
building tree 394 of 400
building tree 395 of 400
building tree 396 of 400
building tree 397 of 400
building tree 398 of 400
building tree 399 of 400
building tree 400 of 400


[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed: 27.1min finished


RandomForestClassifier(max_depth=60, max_features='sqrt', n_estimators=400,
                       n_jobs=-1, random_state=123, verbose=2)

In [75]:
dict_topic = dict(zip(X_train.topic, X_train.topic_le))
keys = list(dict_topic.keys())
keys.sort()
dict_topic_new = {}
for key in keys:
  dict_topic_new[key] = dict_topic[key]
dict_topic_new

{'69-я параллель': 0,
 'Бизнес': 2,
 'Бывший СССР': 3,
 'Дом': 4,
 'Забота о себе': 5,
 'Из жизни': 6,
 'Интернет и СМИ': 7,
 'Крым': 8,
 'Культпросвет ': 9,
 'Культура': 10,
 'Легпром': 11,
 'Мир': 13,
 'Моя страна': 15,
 'Наука и техника': 16,
 'Нацпроекты': 17,
 'Путешествия': 19,
 'Россия': 20,
 'Силовые структуры': 21,
 'Спорт': 23,
 'Среда обитания': 24,
 'Ценности': 25,
 'Экономика': 27}

In [ ]:
from sklearn.metrics import f1_score
pred_train = model.predict(full_X_train)
score_train = f1_score(pred_train, Y_train, sample_weight=None, average="micro")
score_train

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    3.4s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:   12.7s
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:   28.1s
[Parallel(n_jobs=4)]: Done 400 out of 400 | elapsed:   31.4s finished


0.9577436964869931

In [ ]:
pred = model.predict(full_X_test)
score = f1_score(pred, Y_test, sample_weight=None, average="micro")
score

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    2.9s
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:    6.3s
[Parallel(n_jobs=4)]: Done 400 out of 400 | elapsed:    6.9s finished


0.7224765116839316

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn import metrics

print (metrics.classification_report (Y_train, pred_train,target_names = dict_topic_new))

                   precision    recall  f1-score   support

   69-я параллель       1.00      0.27      0.42       604
           Бизнес       1.00      0.65      0.79      1337
      Бывший СССР       0.99      0.91      0.95     16878
              Дом       1.00      0.81      0.89      3984
    Забота о себе       1.00      0.11      0.20       571
         Из жизни       1.00      0.63      0.77      8066
   Интернет и СМИ       0.99      0.79      0.88     11664
             Крым       1.00      0.35      0.52       120
    Культпросвет        1.00      0.30      0.46        61
         Культура       0.97      0.93      0.95     12918
          Легпром       1.00      0.62      0.76        21
              Мир       0.90      0.96      0.93     35240
       Моя страна       1.00      0.40      0.57      1103
  Наука и техника       0.98      0.93      0.95     13610
       Нацпроекты       1.00      0.93      0.96       851
      Путешествия       1.00      0.80      0.89      4

In [ ]:
print (metrics.classification_report(Y_test, pred,target_names = dict_topic_new))

                   precision    recall  f1-score   support

   69-я параллель       0.00      0.00      0.00       121
           Бизнес       0.00      0.00      0.00       268
      Бывший СССР       0.79      0.62      0.69      3376
              Дом       0.86      0.48      0.62       797
    Забота о себе       0.00      0.00      0.00       114
         Из жизни       0.89      0.26      0.40      1613
   Интернет и СМИ       0.81      0.33      0.47      2333
             Крым       0.00      0.00      0.00        24
    Культпросвет        0.00      0.00      0.00        12
         Культура       0.81      0.79      0.80      2584
          Легпром       0.00      0.00      0.00         4
              Мир       0.68      0.83      0.75      7048
       Моя страна       0.00      0.00      0.00       221
  Наука и техника       0.82      0.75      0.78      2722
       Нацпроекты       1.00      0.65      0.79       170
      Путешествия       0.94      0.44      0.60       

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Попробуем применить для подбора гиперпараметров оптуну**

In [66]:
def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 10, 100),
        'n_estimators': trial.suggest_int('n_estimators', 50, 400),
        'min_samples_split': trial.suggest_int('min_samples_split', 1, 150),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 100)
    }

    #smoth_n_neighbors = trial.suggest_int('smoth_n_neighbors', 5, 10)
    #sampler = SMOTE(random_state=42, k_neighbors=smoth_n_neighbors)

    clf = RandomForestClassifier(random_state=42, **params)

    #pipeline = make_pipeline(sampler, clf)


    scores = cross_validate(clf, full_X_train, Y_train, verbose=1,
                    n_jobs=-1, scoring="accuracy", cv=3)
    
    print(scores)
    return scores['test_score'].mean()


In [67]:
studies = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
studies.optimize(objective, n_trials=10)

print(studies.best_value)
print(studies.best_params)

[I 2023-02-26 18:48:46,395] A new study created in memory with name: no-name-32d8e37e-6ebb-461b-84d0-b0e4f4021dab
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed: 11.7min finished
[I 2023-02-26 19:00:29,645] Trial 0 finished with value: 0.6904111437147965 and parameters: {'max_depth': 99, 'n_estimators': 340, 'min_samples_split': 60, 'min_samples_leaf': 14}. Best is trial 0 with value: 0.6904111437147965.


{'fit_time': array([678.3273654 , 682.16300082, 674.60475016]), 'score_time': array([21.34737992, 20.60008097, 23.6046567 ]), 'test_score': array([0.68842601, 0.69095561, 0.69185181])}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  2.4min finished
[I 2023-02-26 19:02:55,933] Trial 1 finished with value: 0.6573095695831707 and parameters: {'max_depth': 88, 'n_estimators': 121, 'min_samples_split': 55, 'min_samples_leaf': 37}. Best is trial 0 with value: 0.6904111437147965.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


{'fit_time': array([125.07641554, 136.89189315, 139.35314441]), 'score_time': array([6.26713061, 5.33424211, 5.44083238]), 'test_score': array([0.65545453, 0.65801304, 0.65846114])}


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  3.8min finished
[I 2023-02-26 19:06:43,195] Trial 2 finished with value: 0.6096086112275528 and parameters: {'max_depth': 38, 'n_estimators': 334, 'min_samples_split': 57, 'min_samples_leaf': 68}. Best is trial 0 with value: 0.6904111437147965.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


...

{'fit_time': array([103.65283656, 106.07562089, 104.05819368]), 'score_time': array([13.20844817, 14.19990253, 13.09107161]), 'test_score': array([0.53806681, 0.53735852, 0.53874619])}


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed: 13.2min finished
[I 2023-02-26 19:30:17,858] Trial 8 finished with value: 0.7093517969770119 and parameters: {'max_depth': 73, 'n_estimators': 241, 'min_samples_split': 143, 'min_samples_leaf': 1}. Best is trial 8 with value: 0.7093517969770119.


{'fit_time': array([781.06743574, 769.03730321, 767.69127631]), 'score_time': array([13.33371139, 17.56689024, 18.60775208]), 'test_score': array([0.7079545 , 0.710513  , 0.70958789])}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/usr/local/lib/python3.8/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  3.6min finished
[I 2023-02-26 19:33:55,232] Trial 9 finished with value: 0.6810733197457877 and parameters: {'max_depth': 97, 'n_estimators': 113, 'min_samples_split': 37, 'min_samples_leaf': 18}. Best is trial 8 with value: 0.7093517969770119.


{'fit_time': array([203.81914425, 210.61638236, 201.56714892]), 'score_time': array([5.52138138, 5.27939463, 8.19047618]), 'test_score': array([0.6807505 , 0.68141542, 0.68105405])}
0.7093517969770119
{'max_depth': 73, 'n_estimators': 241, 'min_samples_split': 143, 'min_samples_leaf': 1}


In [68]:
print('Number of finished trials: {}'.format(len(studies.trials)))
print('Best trial:')
trial = studies.best_trial

print('  Value: {}'.format(trial.value))
print('  Params: ')

for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

Number of finished trials: 10
Best trial:
  Value: 0.7093517969770119
  Params: 
    max_depth: 73
    n_estimators: 241
    min_samples_split: 143
    min_samples_leaf: 1


Отметим, что и оптюна подобрала параметры близкие к нашим изначальным по глубине деревев и их количеству

In [69]:
from optuna.visualization import plot_slice
plot_slice(studies)

In [71]:
Y_train

0         20
1          3
2          7
3         20
4         25
          ..
207538     6
207539     3
207540    13
207541    13
207542    27
Name: topic_le, Length: 207543, dtype: int64

In [72]:
Y_test

0        20
1        13
2        13
3        13
4        27
         ..
41505    13
41506    23
41507    13
41508    20
41509    20
Name: topic_le, Length: 41510, dtype: int64

In [70]:
model_random_forest= RandomForestClassifier(
    **studies.best_params,
    random_state=42
)
model_random_forest.fit(full_X_train, Y_train)

ValueError: ignored

In [74]:
y_pred = model_random_forest.predict(full_X_test)
score = f1_score(y_pred, Y_test, sample_weight=None, average="micro")
score

0.7185979282100698

In [14]:
dict_topic = dict(zip(X_train.topic, X_train.topic_le))
keys = list(dict_topic.keys())
keys.sort()
dict_topic_new = {}
for key in keys:
  dict_topic_new[key] = dict_topic[key]
dict_topic_new

{'69-я параллель': 0,
 'Бизнес': 2,
 'Бывший СССР': 3,
 'Дом': 4,
 'Забота о себе': 5,
 'Из жизни': 6,
 'Интернет и СМИ': 7,
 'Крым': 8,
 'Культпросвет ': 9,
 'Культура': 10,
 'Легпром': 11,
 'Мир': 13,
 'Моя страна': 15,
 'Наука и техника': 16,
 'Нацпроекты': 17,
 'Путешествия': 19,
 'Россия': 20,
 'Силовые структуры': 21,
 'Спорт': 23,
 'Среда обитания': 24,
 'Ценности': 25,
 'Экономика': 27}

In [78]:
from sklearn.metrics import accuracy_score
from sklearn import metrics
print (metrics.classification_report(Y_test,y_pred,target_names = dict_topic_new))

                   precision    recall  f1-score   support

   69-я параллель       0.00      0.00      0.00       121
           Бизнес       0.00      0.00      0.00       268
      Бывший СССР       0.78      0.65      0.71      3376
              Дом       0.83      0.46      0.59       797
    Забота о себе       0.00      0.00      0.00       114
         Из жизни       0.85      0.35      0.50      1613
   Интернет и СМИ       0.80      0.36      0.50      2333
             Крым       0.00      0.00      0.00        24
    Культпросвет        0.00      0.00      0.00        12
         Культура       0.80      0.81      0.81      2584
          Легпром       0.00      0.00      0.00         4
              Мир       0.69      0.83      0.76      7048
       Моя страна       0.00      0.00      0.00       221
  Наука и техника       0.81      0.77      0.79      2722
       Нацпроекты       0.99      0.62      0.77       170
      Путешествия       0.92      0.52      0.67       

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [81]:
import pickle
with open('/content/drive/Othercomputers/My Laptop/Project_1_HSE/PIKL/random_forest_all_feature.pkl', 'wb') as f:
          pickle.dump({
               'model_rf_all_feature':model_random_forest
           },f)

**Сделаем отбор признаков с помощью логрега**

In [8]:
#Закодируем признаки связанные с датами.
enc = OneHotEncoder(drop='first', handle_unknown='ignore')

enc.fit(X_train[['day_of_week','seanos', 'year']])

train_num = enc.transform(X_train[['day_of_week','seanos', 'year']])
test_num = enc.transform(X_test[['day_of_week','seanos', 'year']])

In [21]:
X_train_bow = vec.fit_transform(X_train['text_str'])

In [22]:
X_test_bow = vec.transform(X_test['text_str'])

In [11]:
# Соеденим данные с датами и текстом в спарс матрицу
full_X_train_ohe = scipy.sparse.hstack([X_train_bow, train_num]).tocsr()
full_X_test_ohe = scipy.sparse.hstack([X_test_bow, test_num]).tocsr()

In [11]:
full_X_train_ohe

<207543x34230 sparse matrix of type '<class 'numpy.float64'>'
	with 20644652 stored elements in Compressed Sparse Row format>

Первый вариант solver='saga'
Оказался слишком долгим в расчете (4 часа)

---



In [15]:
%%time

clf = LogisticRegression(random_state=42, max_iter=300, warm_start=True, solver='saga', n_jobs=-1, penalty='l1')

clf.fit(full_X_train_ohe, Y_train)

CPU times: user 4h 57min 54s, sys: 9.17 s, total: 4h 58min 3s
Wall time: 1h 40min 41s


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(max_iter=300, n_jobs=-1, penalty='l1', random_state=42,
                   solver='saga', warm_start=True)

In [16]:
import pickle
with open('/content/drive/Othercomputers/My Laptop/Project_1_HSE/PIKL/LogisticRegression.pkl', 'wb') as f:
          pickle.dump({
               'logreg_model':clf
           },f)

In [20]:
with open('/content/drive/Othercomputers/My Laptop/Project_1_HSE/PIKL/LogisticRegression.pkl', 'rb') as f:
         data = pickle.load(f)
clf = data['logreg_model']

In [21]:
sel = SelectFromModel(clf, prefit=True, threshold=0.1)

X_train_ohe_new = sel.transform(full_X_train_ohe)
X_train_ohe_new.shape

(207543, 7751)

Втрой вариант solver='sag'. Тут можно применить только L2 регуляризацию. потому будут отбрасываться признаки меньшие среднего ( по умолчанию в гиперах стоит  threshold = mean)

In [ ]:
%%time

clf_1 = LogisticRegression(random_state=42, max_iter=500, warm_start=True, solver='sag', n_jobs=-1,verbose=2 )
clf_1.fit(full_X_train_ohe, Y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


convergence after 178 epochs took 558 seconds
CPU times: user 9min 20s, sys: 434 ms, total: 9min 20s
Wall time: 9min 18s


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  9.3min finished


LogisticRegression(max_iter=500, n_jobs=-1, random_state=42, solver='sag',
                   verbose=2, warm_start=True)

In [31]:
#Сохраним вторую модель логрега
import pickle
with open('/content/drive/Othercomputers/My Laptop/Project_1_HSE/PIKL/LogisticRegression_1.pkl', 'wb') as f:
          pickle.dump({
               'logreg_model':clf_1
           },f)

In [23]:
sel = SelectFromModel(clf_1, prefit=True)

X_train_ohe_new = sel.transform(full_X_train_ohe)
X_train_ohe_new.shape

(207543, 10500)

In [33]:
X_train_ohe_new

<207543x10500 sparse matrix of type '<class 'numpy.float64'>'
	with 18717323 stored elements in Compressed Sparse Row format>

In [24]:
X_test_ohe_new = sel.transform(full_X_test_ohe)
X_test_ohe_new.shape

(41510, 10500)

In [34]:
#Сохраним датасеты с уменьшенным количеством признаков
import pickle
with open('/content/drive/Othercomputers/My Laptop/Project_1_HSE/PIKL/X_reduce.pkl', 'wb') as f:
          pickle.dump({
               'X_train':X_train_ohe_new,
               'X_test':X_test_ohe_new,
               'Y_train':Y_train,
               'Y_test':Y_test
           },f)

In [6]:
with open('/content/drive/Othercomputers/My Laptop/Project_1_HSE/PIKL/X_reduce.pkl', 'rb') as f:
         data = pickle.load(f)
X_train_ohe_new = data['X_train']
X_test_ohe_new = data['X_test']
Y_train = data['Y_train']
Y_test = data['Y_test']

In [9]:
X_test_ohe_new

<41510x10500 sparse matrix of type '<class 'numpy.float64'>'
	with 3763664 stored elements in Compressed Sparse Row format>

Применим оптуну еще раз уже для датасета с меньшим размером. Потому решено было увеличить количество деревьев.Но что и тербовалось доказать большое количество деревьев для леса не играет роли

In [14]:
def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 10, 100),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'min_samples_split': trial.suggest_int('min_samples_split', 1, 150),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 100)
    }

    clf = RandomForestClassifier(random_state=42, **params)

    clf.fit(X_train_ohe_new, Y_train)

    # Make predictions
    y_pred = clf.predict(X_test_ohe_new)

    # Evaluate predictions
    score = f1_score(y_pred, Y_test, average="micro")

    return score


In [15]:
%%time
studies = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
studies.optimize(objective, n_trials=20)

[I 2023-02-28 14:26:33,065] A new study created in memory with name: no-name-c1b8c4b9-d855-46eb-a416-258bcbe3f101
[I 2023-02-28 14:34:58,860] Trial 0 finished with value: 0.6493134184533847 and parameters: {'max_depth': 79, 'n_estimators': 351, 'min_samples_split': 19, 'min_samples_leaf': 100}. Best is trial 0 with value: 0.6493134184533847.
[I 2023-02-28 14:55:12,435] Trial 1 finished with value: 0.6475307154902433 and parameters: {'max_depth': 44, 'n_estimators': 815, 'min_samples_split': 88, 'min_samples_leaf': 71}. Best is trial 0 with value: 0.6493134184533847.
[I 2023-02-28 15:01:16,017] Trial 2 finished with value: 0.6692122380149361 and parameters: {'max_depth': 64, 'n_estimators': 168, 'min_samples_split': 45, 'min_samples_leaf': 49}. Best is trial 2 with value: 0.6692122380149361.
[I 2023-02-28 15:12:29,820] Trial 3 finished with value: 0.6661286437003132 and parameters: {'max_depth': 94, 'n_estimators': 341, 'min_samples_split': 64, 'min_samples_leaf': 61}. Best is trial 2 w

CPU times: user 12h 37min 32s, sys: 44.8 s, total: 12h 38min 17s
Wall time: 12h 35min 4s


In [16]:
print('Number of finished trials: {}'.format(len(studies.trials)))
print('Best trial:')
trial = studies.best_trial

print('  Value: {}'.format(trial.value))
print('  Params: ')

for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

Number of finished trials: 20
Best trial:
  Value: 0.7318477475307155
  Params: 
    max_depth: 97
    n_estimators: 993
    min_samples_split: 120
    min_samples_leaf: 1


In [17]:
model_random_forest= RandomForestClassifier(
    **studies.best_params,
    random_state=42
)
model_random_forest.fit(X_train_ohe_new, Y_train)

RandomForestClassifier(max_depth=97, min_samples_split=120, n_estimators=993,
                       random_state=42)

In [18]:
import pickle
with open('/content/drive/Othercomputers/My Laptop/Project_1_HSE/PIKL/random_forest_reduce_feature.pkl', 'wb') as f:
          pickle.dump({
               'model_rf_reduce_feature':model_random_forest
           },f)

In [20]:
y_pred = model_random_forest.predict(X_test_ohe_new)
score = f1_score(y_pred, Y_test, sample_weight=None, average="micro")
score

0.7318477475307155

Попробуем для этого же датасета ( с уменьшенным количеством фичей) уменьшить количество деревьев.
И получается тот же скор. так что оокончательно примем следующие гиперпараметры.
Верно, что с увеличением числа деревьев скор не будет сильно увеличиваться. Можно было для отпуны задать и меньшее количество деревьев.

In [22]:
trial.params

{'max_depth': 97,
 'n_estimators': 993,
 'min_samples_split': 120,
 'min_samples_leaf': 1}

In [8]:
model_random_forest = RandomForestClassifier(
  n_estimators=300,
  max_depth = 97,
  bootstrap=True,
  min_samples_split = 120,
  random_state=42,
  n_jobs=-1
)

# Обучаем на тренировочных данных
model_random_forest.fit(X_train_ohe_new, Y_train)

y_pred = model_random_forest.predict(X_test_ohe_new)
score = f1_score(y_pred, Y_test, sample_weight=None, average="micro")
score

0.730281859792821

In [16]:
import pickle
with open('/content/drive/Othercomputers/My Laptop/Project_1_HSE/PIKL/random_forest_reduce_feature_best.pkl', 'wb') as f:
          pickle.dump({
               'model_rf_reduce_feature)best':model_random_forest
           },f)

In [15]:
from sklearn.metrics import accuracy_score
from sklearn import metrics
print (metrics.classification_report(Y_test,y_pred,target_names = dict_topic_new))

                   precision    recall  f1-score   support

   69-я параллель       0.00      0.00      0.00       121
           Бизнес       0.00      0.00      0.00       268
      Бывший СССР       0.77      0.71      0.74      3376
              Дом       0.81      0.50      0.62       797
    Забота о себе       0.93      0.12      0.22       114
         Из жизни       0.79      0.40      0.54      1613
   Интернет и СМИ       0.78      0.41      0.53      2333
             Крым       0.00      0.00      0.00        24
    Культпросвет        0.00      0.00      0.00        12
         Культура       0.80      0.83      0.81      2584
          Легпром       0.00      0.00      0.00         4
              Мир       0.70      0.83      0.76      7048
       Моя страна       1.00      0.00      0.01       221
  Наука и техника       0.79      0.78      0.79      2722
       Нацпроекты       0.97      0.68      0.80       170
      Путешествия       0.89      0.55      0.68       

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Итоговый скор на тестовых данных - 0.73**


Итоги


*   Random forest - модель,самая долгая по времени расчета. Было сделано уменьшение признаков в матрице данных и так же применена оптюна. Оптюна не подобрала лучших гиперов для модели и соотвеннно скор не повысился.
*   Скор ниже, чем у других моделей в проекте.Возможно малочисленные классы попадают в другие классы при предсказнии классов в силу похожести тем. 
